Schaeffer and Tran 2020

## 1. Import Packages and Define Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import spgl1
from spgl1 import spgl1, spg_lasso, spg_bp, spg_bpdn, spg_mmv

#==============================================================
# Activation Function
def msin(x):
    return np.sin(x)


#==============================================================
# Target Functions
def fn1(x):
    s = 0
    for i in range(len(x)-1):
        s += np.exp(-x[i]**2)/(1+x[i+1]**2)
    
    return s/len(x)


In [2]:
# Example 1 ( See Figure 1, log of training size = 2.4, q = 2, log of error of SRF ~ -1.6) 
ED = 250 # training set size (number of measurements m in the paper)
n = ED + 5000 # total dataset size (training + validation)
d = 10 # ambient dimension 
q = 2 # dimension of low order features
N = 5000 # number of features
omega_weight = 1 # scale for the random weights
sigma_spgl1 = 1e-2 #s igma_spgl1 = eta* sqrt(m), stability parameter for the B.P. problem
mactivation = msin
Testing_Func = fn1

In [3]:
 # basis pursuit denoising: \|y - Ac \|_2 <= sigma_spgl1 
iterMax_spgl1 = 5000 # maximum number of iterations for BP
Runs = 1 # number of random trials
error_train_vec_srf = np.zeros(Runs)
error_test_vec_srf = np.zeros(Runs)

tol_dict = 1e-10
sparsity_percentage_vec = np.zeros(Runs)
for i in range(Runs):
    # Create input-output data
    data_input = 2*np.random.rand(n,d)-1
    data_output = np.zeros(n)
    for j in range(n):
        data_output[j] = Testing_Func(data_input[j,:])
    

    # Create dictionary matrix
    
    idx = 0
    A = np.zeros((n,N))

    tmp = 1/np.sqrt(ED)*np.ones(data_input.shape[0])
    A[:,0] = tmp

    listind =[]
    for idx in range(1,N):
        w = omega_weight*np.random.randn(q)
        p = np.pi*2*np.random.rand(1)

        xind = np.random.choice(d,q,replace = False)


        tmp = mactivation(np.dot(data_input[:,xind],w) + p)
        
        scale = np.sqrt(np.dot(tmp[:ED],tmp[:ED])) 

        if scale>(np.sqrt(n)*tol_dict):
            tmp /=scale
            A[:,idx] = tmp
            
    # Create train and test
   

    A_train = A[:ED,:]
    A_test = A[ED:,:]
    y_train = data_output[:ED]
    y_test = data_output[ED:]
    c,resid,grad,info = spg_bpdn(A_train, y_train, sigma = sigma_spgl1, iter_lim= iterMax_spgl1, verbosity=0)
    

    
    y_train_model = np.dot(A_train, c) 
 
    error_train = np.linalg.norm(data_output[:ED] - y_train_model)/np.linalg.norm(data_output[:ED])
    
  
    y_test_model= np.dot(A_test, c)
    error_test = np.linalg.norm(y_test - y_test_model)/np.linalg.norm(y_test)
    error_train_vec_srf[i] = error_train
    error_test_vec_srf[i] = error_test

print("Average of Training Error (percentage): ", np.mean(error_train_vec_srf)*100)
print("Average of Testing Error (percentage): ", np.mean(error_test_vec_srf)*100)
print("Log of Training Size vs Log of Testing Error: ", [np.log10(ED), np.log10(error_test_vec_srf)])


Linesearch failed with error 1. Damping max BB scaling to 10000.0
Linesearch failed with error 1. Damping max BB scaling to 1000.0
Linesearch failed with error 1. Damping max BB scaling to 100.0
Linesearch failed with error 1. Damping max BB scaling to 10.0
Linesearch failed with error 1. Damping max BB scaling to 1.0
Linesearch failed with error 1. Damping max BB scaling to 0.1


Average of Training Error (percentage):  0.4152063025120786
Average of Testing Error (percentage):  2.3514007459572985
Log of Training Size vs Log of Testing Error:  [2.3979400086720375, array([-1.62867335])]


## 